In [1]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

#Loads the VGG16 model 
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

Using TensorFlow backend.


In [2]:
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " " + layer.__class__.__name__ + "\t" ,layer.trainable)

0 InputLayer	 False
1 Conv2D	 True
2 Conv2D	 True
3 MaxPooling2D	 True
4 Conv2D	 True
5 Conv2D	 True
6 MaxPooling2D	 True
7 Conv2D	 True
8 Conv2D	 True
9 Conv2D	 True
10 MaxPooling2D	 True
11 Conv2D	 True
12 Conv2D	 True
13 Conv2D	 True
14 MaxPooling2D	 True
15 Conv2D	 True
16 Conv2D	 True
17 Conv2D	 True
18 MaxPooling2D	 True


In [3]:
for layer in model.layers:
    layer.trainable = False
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " " + layer.__class__.__name__ + "\t" ,layer.trainable)

0 InputLayer	 False
1 Conv2D	 False
2 Conv2D	 False
3 MaxPooling2D	 False
4 Conv2D	 False
5 Conv2D	 False
6 MaxPooling2D	 False
7 Conv2D	 False
8 Conv2D	 False
9 Conv2D	 False
10 MaxPooling2D	 False
11 Conv2D	 False
12 Conv2D	 False
13 Conv2D	 False
14 MaxPooling2D	 False
15 Conv2D	 False
16 Conv2D	 False
17 Conv2D	 False
18 MaxPooling2D	 False


# joining Top and Bottom of models

In [4]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [5]:
model.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 3
FC_Head = addTopModel(model, num_classes)
modelnew = Model(inputs=model.input, outputs=FC_Head)

In [7]:
print(modelnew.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [8]:
from keras.preprocessing.image import ImageDataGenerator

In [9]:
#I haven't done data augmentation here because I already created images using a my own augmentation script
# It has variety of options if ypou want to check I have addded with this Python File

In [10]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="/root/mlopsproject/UntitledFolder/Traning2/",target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="/root/mlopsproject/UntitledFolder/Validation2/", target_size=(224,224))

Found 916 images belonging to 3 classes.
Found 106 images belonging to 3 classes.


In [11]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [12]:
checkpoint = ModelCheckpoint("friends_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 4,
                          verbose = 1,
                          restore_best_weights = False)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 916
nb_validation_samples = 106
epochs = 6
batch_size = 8

history = modelnew.fit_generator(
    traindata,
    steps_per_epoch = nb_validation_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = testdata,
    validation_steps = nb_validation_samples // batch_size
)

modelnew.save("friends_vgg16f.h5")

Epoch 1/6
13/13 [==============================] - 190s 15s/step - loss: 52.5511 - accuracy: 0.5529 - val_loss: 22.7194 - val_accuracy: 0.5771

Epoch 00001: val_loss improved from inf to 22.71938, saving model to friends_vgg.h5
Epoch 2/6
13/13 [==============================] - 184s 14s/step - loss: 11.3474 - accuracy: 0.7284 - val_loss: 12.9170 - val_accuracy: 0.6543

Epoch 00002: val_loss improved from 22.71938 to 12.91702, saving model to friends_vgg.h5
Epoch 3/6
13/13 [==============================] - 179s 14s/step - loss: 2.7683 - accuracy: 0.8418 - val_loss: 11.3336 - val_accuracy: 0.5714

Epoch 00003: val_loss improved from 12.91702 to 11.33361, saving model to friends_vgg.h5
Epoch 4/6
13/13 [==============================] - 180s 14s/step - loss: 2.6902 - accuracy: 0.8341 - val_loss: 0.1438 - val_accuracy: 0.8963

Epoch 00004: val_loss improved from 11.33361 to 0.14382, saving model to friends_vgg.h5
Epoch 5/6
13/13 [==============================] - 200s 15s/step - loss: 0.47

In [13]:
traindata.class_indices

{'Akshansh': 0, 'Aman': 1, 'Ritesh': 2}

In [15]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join


singers_dict = { 
    0:'Akshansh',
    1:'Aman',
    2:'Ritesh',
}


def draw_test(name, pred, im):
    singer = singers_dict[pred]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, singer, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + path_class)
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,20):
    input_im = getRandomImage("/root/mlopsproject/UntitledFolder/Testing2/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(modelnew.predict(input_im))
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

Class - Aman
Class - Aman
Class - Akshansh
Class - Akshansh
Class - Akshansh
Class - Aman
Class - Akshansh
Class - Ritesh
Class - Akshansh
Class - Akshansh
Class - Akshansh
Class - Ritesh
Class - Ritesh
Class - Ritesh
Class - Aman
Class - Aman
Class - Aman
Class - Akshansh
Class - Akshansh
Class - Ritesh
